## Correct illumination using BaSiC algorithm 

Illumination pattern is caclulated for each channel (all wells together). 

For an experiment imaging entire wells run on CTRL or limit the number of tiles to calculate the illumination pattern.

In [1]:
import os
import random
from tifffile import imwrite, imread
from readlif.reader import LifFile
import napari
import numpy as np
from dask import delayed
import dask.array as da

from basicpy import BaSiC

In [2]:
# define input and output pathway
im_dir = r'' # pathway to the directory containing tiled images 

save_dir = r'' # pathway to the output directory where corrected tiles will be stored  
save_basic_dir = r'' # pathway to store illumination patterns for quality control 

# experiment rounds to be analyzed
round_list = [] # example ['00','01']

# specify how many tiles will be used to calculate illumination pattern
# if set to 0, the illumination pattern will be calculated from the entire image
tile_number = 0

In [ ]:
for r in round_list:

    channel_list  = set([f[f.index('ch_')+3:][:2] for f in os.listdir(im_dir) if (f'r_{r}' in f)])

    for ch in channel_list:

        print(f'Processing round {r}, channel {ch}.')

        im_files = [os.path.join(im_dir, f) for f in os.listdir(im_dir) if (f'r_{r}_ch_{ch}' in f)]

        # read in images
        im_list = []
        for im_name in im_files:
            im = imread(im_name)
            im_list.append(im)

        # select signal images
        im_sel = []
        name_sel = []
        for im,im_name in zip(im_list,im_files):
            if (np.max(im)>0):
                im_sel.append(im)
                name_sel.append(im_name)
            else:
                imwrite(os.path.join(save_dir, os.path.basename(im_name)), im.astype('uint16'))
        
        im_test = np.array(im_sel)

        # remove high signal artifacts
        im_pattern_list = []
        for im in im_sel:
            if (np.percentile(im,99)<(2**16-1)):
                im_pattern_list.append(im)

        im_pattern = np.array(im_pattern_list)

        # find illumination pattern with BaSiC
        basic = BaSiC(get_darkfield=False, smoothness_flatfield=1, smoothness_darkfield=1)
        if tile_number > 0:
            basic.fit(np.array(random.sample(im_pattern_list,tile_number)))
        else:
            basic.fit(im_pattern)

        images_transformed = basic.transform(im_test)

        # save basic fields
        imwrite(os.path.join(save_basic_dir, f'r_{r}_ch_{ch}_flatfield_basic.tif'), basic.flatfield)
        imwrite(os.path.join(save_basic_dir, f'r_{r}_ch_{ch}_darkfield_basic.tif'), basic.darkfield)

        # save tiles
        for im,im_name in zip(images_transformed,name_sel):
            imwrite(os.path.join(save_dir, os.path.basename(im_name)), im.astype('uint16'))
